In [1]:
import pandas as pd
import os
from pathlib import Path
from PIL import Image
from img2vec_pytorch import Img2Vec
import numpy as np
import pinecone
import codecs
import json
import tqdm

from torchvision import transforms as ts 
import torchvision.models as models

c:\Users\mohit\anaconda3\envs\snowpark\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [37]:
main_path='./static/img/'
depts=[]
genders=[]
img_paths=[]

for chromo in os.listdir(main_path):
    dept_path=main_path+chromo
    if ".jpg" not in dept_path:
        for dept in os.listdir(dept_path):
            dept_img_path=dept_path+"/"+dept
            for imgs in os.listdir(dept_img_path):
                    img_path=dept_img_path+"/"+imgs
                    img_paths.append(img_path)
                    depts.append(dept)
                    genders.append(chromo)

In [22]:
main_path='./static/img/women/'
depts=[]
genders=[]
img_paths=[]
for dept in os.listdir("./static/img/women/"):
        dept_img_path=main_path+"/"+dept
        for imgs in os.listdir(dept_img_path):
                img_path=dept_img_path+"/"+imgs
                img_paths.append(img_path)
                depts.append(dept)

In [3]:
# img_paths=[]
# for img_path in sorted(Path("./static/img/women/cardigan").glob("*.jpg")):
#         img_paths.append(str(img_path))
# print(len(img_paths))

101


In [8]:
class ImageEmbedder:
    def __init__(self):
        self.normalize=ts.Normalize(
            mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]
        )
        self.model=models.squeezenet1_0(pretrained=True,progress=False)

    def embed(self,image_file_name):
        image=Image.open(image_file_name)
        image=ts.Resize(256)(image)
        image=ts.CenterCrop(224)(image)
        tensor=ts.ToTensor()(image)
        tensor=self.normalize(tensor).reshape(1,3,224,224)
        vector=self.model(tensor).cpu().detach().numpy().flatten().tolist()
        return vector
    
image_embedder= ImageEmbedder()

c:\Users\mohit\anaconda3\envs\snowpark\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mohit\anaconda3\envs\snowpark\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
df=pd.DataFrame(img_paths,columns=['image_file_name'])
df['embedding']=[
    image_embedder.embed(file_name) for file_name in img_paths
]

In [44]:
json_dat=[]
for id,values,gender,dept in zip(df.image_file_name,df.embedding,genders,depts):
    dic={'id':id,'values':values,'metadata':{'gender':gender,'dept':dept}}
    json_dat.append(dic)

In [45]:

pinecone.init('18688bbc-9aa5-4be2-a87d-29c18cede436',environment='us-west4-gcp')
# pinecone.create_index('img-vectors',dimension=1000)

INDEX_NAME='img-vector'

pinecone.create_index(INDEX_NAME,dimension=1000,metric="euclidean",shards=1)
index=pinecone.Index(INDEX_NAME)

In [56]:
index.upsert(vectors=json_dat[0:500])
index.upsert(vectors=json_dat[500:])

{'upserted_count': 439}

In [29]:
vectors=image_embedder.embed(r'static\uploaded\Shirts_Polos679.jpg')
output=index.query(vectors,top_k=5,include_metadata=True)
print(output.get('matches'))

[{'id': './static/img/women//Jackets_Coats/Jackets_Coats67.jpg',
 'metadata': {'dept': 'Jackets_Coats', 'gender': 'WOMEN'},
 'score': 2488.67969,
 'values': []}, {'id': './static/img/women//Dresses/Dresses4.jpg',
 'metadata': {'dept': 'Dresses', 'gender': 'WOMEN'},
 'score': 2599.88281,
 'values': []}, {'id': './static/img/women//Dresses/Dresses96.jpg',
 'metadata': {'dept': 'Dresses', 'gender': 'WOMEN'},
 'score': 2966.92969,
 'values': []}, {'id': './static/img/women//Cardigans/Cardigans90.jpg',
 'metadata': {'dept': 'Cardigans', 'gender': 'WOMEN'},
 'score': 3013.84375,
 'values': []}, {'id': './static/img/women//Jackets_Coats/Jackets_Coats76.jpg',
 'metadata': {'dept': 'Jackets_Coats', 'gender': 'WOMEN'},
 'score': 3044.28906,
 'values': []}]


In [17]:
match=output.get('matches')
match[0].get('id')

'static\\img\\women\\cardigan\\Cardigans89.jpg'